In [1]:
import numpy as np
def V_harm (x):
    return 0.5*x**2
def V_anharm (x,lamb):
    return 0.5*x**2 + lamb*x**4
def action (path, beta, V_function):
    P = path.shape[0]
    dt = beta/P
    diffs = path - np.roll(path,1)
    S = np.sum(diffs**2) / (2.0 * dt)
    E = dt * np.sum(V_function(path))
    return S + E
def MC_1D (V_function, beta, P, n_steps=200000, burn_in = 2000, step_scale =0.5,seed=1):
    rng = np.random.default_rng(seed)
    path = np.zeros(P, dtype= float)
    accepted = 0
    measurements = []
    S_current = action (path,beta,V_function)
    for i in range (n_steps + burn_in):
        k = rng.integers(0, P)
        old = path [k]
        new = old + rng.normal(0.0, step_scale)
        path[k]=new
        S_new = action (path, beta, V_function)
        ln_alpha = S_current - S_new
        if np.log(rng.random()) < ln_alpha:
            S_current = S_new
            accepted += 1
        else:
            path[k] = old
        if i  >= burn_in:
            measurements.append(np.mean(path**2))
    accrate = accepted / n_steps
    return np.array(measurements), accrate
meas_1, acc_1 = MC_1D(V_harm, 2, 100)
print("Harmonic: acc =", acc_1, " <x^2> =", meas_1.mean())
lamb = 0.1
V_a = lambda x: V_anharm(x, lamb=lamb)
meas_2,acc_2 = MC_1D(V_a, 2, 100)
print("Anhrmonic: acc =", acc_2, " <x^2> =", meas_2.mean())




Harmonic: acc = 0.245115  <x^2> = 0.0955597214526498
Anhrmonic: acc = 0.246035  <x^2> = 0.0823570579464059
